# Q-Learning to solve the Taxi-v2 Environment

This notebook partially follows, and uses the same illustration as "Reinforcement Q-Learning from Scratch in Python with OpenAI Gym by LearnDataSci" for the sake of comparison, but does not use the same code.

Q-learning lets the agent use the environment's rewards to learn, the best action to take in a given state, over time.<br>
In the Taxi environment, we have the reward table, that the agent will learn from. It does this by looking receiving a reward for taking an action in the current state, then updating a value to remember if that action was beneficial.<br>
These values are known as Q-values and are stored in a Q-table. They map to a (state, action) combination.<br>
A Q-value for a particular state-action combination is representative of the "quality" of an action taken from that state. Better Q-values imply better chances of getting greater rewards.<br>
Q-values are initialized to an arbitrary value, and as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-values are updated. This is the Q-Learning algorithm.<br>
So,
1. The Q-table is initialized by all zeros.
2. For each state (S), one action among all possible actions is selected.
3. (S') is the outcome state as a result of that action (a).
4. For all possible actions in the state (S') the one with the highest Q-value is selected.
5. Q-table values are updated using the equation.
6. the new state is set as the current state
7. Once the goal is reached, end and repeat the process for the next episode.

In [118]:
import gym
env = gym.make("Taxi-v2").env
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



The State Space is the set of all possible situations our taxi could inhabit, whereas the Action Space is the set of all the actions that the agent can take in a given state.<br>
Since there are 5 X 5 possible taxi locations, 5 possible passenger locations (4 corresponding to RGBY with 1 additional of being in the taxi), and 4 possible destinations(corresponding to RGBY), the State Space = 5 X 5 X 5 X 4 = 500.<br>
Since the agent can take 6 possible actions at any given state, the Action Space = 6, namely:
0. south
1. north
2. east
3. west
4. pickup
5. dropoff

In [119]:
env.reset() # reset environment to a new, random state
env.render()

#No. of possible actions
action_size = env.action_space.n
print("Action Space", env.action_space.n)
#No. of possible states
space_size = env.action_space.n
print("State Space", env.observation_space.n)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space 6
State Space 500


The same illustration as in the tutorial is used. Since, the taxi is at row 3, column 1, the passenger is at location 2, and the destination is location 0, the state can be encoded. This particular state will also be used to check the final solution.

In [120]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



When the Taxi environment is created, there is an initial Reward table that's also created. It is a matrix that has the number of states as rows and number of actions as columns. It has the structure {action: (probability, nextstate, reward, done)}.
Since every state is in this matrix, we can see the default reward values assigned to state 328.

In [121]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

The Q-table is a matrix which has a row for every state (500) and a column for every action (6). It is responsible to store score values for each (state, action) pair. It's first initialized to 0, and then values are updated after training, which helps us discover which actions lead to a better stream of rewards. It must be noted that the Q-table has the same dimensions as the reward table, but it has a completely different purpose.

In [122]:
import numpy as np
qtable = np.zeros((env.observation_space.n, env.action_space.n))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


The training algorithm that will update this Q-table as the agent explores the environment over thousands of episodes.

First the hyperparameters are initialized:<br>
alpha is the learning rate.<br>
gamma is the discounting rate.<br>
epsilon is the exploration rate. Since we want less and less exploration over time we reduce the exploration probability over time, keeping it between max_epsilon and min_epsilon.<br>
decay_rate is the exponential decay rate for exploration probability.<br>
Steps refers to maximum no. of steps in one episode.<br>

For each epsiode, it is first decided whether to pick a random action or to exploit the already computed Q-values. This is done simply by using the epsilon value and comparing it to the random.uniform(0, 1) function, which returns an arbitrary number between 0 and 1.

We execute the chosen action in the environment to obtain the new_state and the reward from performing the action. After that, we calculate the maximum Q-value for the actions corresponding to the new_state, and with that, we can easily update our Q-value to the new Q-value:

In [145]:
#Training

import random

steps = 99
total_episodes = 100000     
alpha = 0.618               
gamma = 0.3                

epsilon = 1.0                 
max_epsilon = 1.0             
min_epsilon = 0.000001            
decay_rate = 0.001             


# For plotting metrics
all_epochs = []
all_penalties = []

for episode in range(total_episodes):

    state = env.reset()
    step = 0
    done = False
    
    for step in range(steps):        
        
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(qtable[state,:]) # Exploit learned values
        
        
        # Take the action and observe the outcome state and reward
        new_state, reward, done, info = env.step(action)

        # Q(old):= Q(old) + lr [R + gamma * max Q(new) - Q(old)]
        qtable[state, action] = qtable[state, action] + alpha * (reward + gamma * 
                                    np.max(qtable[new_state, :]) - qtable[state, action])
                
        # Let new_state be state
        state = new_state
        
        # If done : finish episode
        if done == True: 
            break
    
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    
if episode % 100 == 0:
    clear_output(wait=True)

print("Training finished.")

Training finished.


On checking the Q-Table for the above illustration, it can be observed that the maximum Q-value is for "north" (-1.427), so it seems that our Q-learning algorithm has effectively learned the best action to take in the particular setting.

In [148]:
qtable[328]

array([ -1.42851828,  -1.42798094,  -1.42851828,  -1.42839428,
       -10.42839428, -10.42839428])

After 100000 episodes, the obtained Q-Table can reliably be used to play the Taxi Environment, maximising the rewards and minimizing the penalties.<br>
For 100 test epsiodes, where the maximum steps per episode are 99, we achieve a pretty good score over time of 8.84.

In [155]:
#Evaluation

env.reset()
rewards = []
test_episodes = 100

for episode in range(test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(steps):
        
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/test_episodes))

Score 12
Score 14
Score 7
Score 11
Score 10
Score 7
Score 15
Score 9
Score 7
Score 6
Score 5
Score 10
Score 9
Score 7
Score 13
Score 13
Score 5
Score 3
Score 10
Score 9
Score 8
Score 8
Score 15
Score 10
Score 11
Score 6
Score 12
Score 5
Score 9
Score 12
Score 12
Score 12
Score 5
Score 10
Score 11
Score 10
Score 8
Score 7
Score 6
Score 10
Score 10
Score 3
Score 5
Score 9
Score 8
Score 9
Score 11
Score 9
Score 7
Score 9
Score 8
Score 9
Score 7
Score 6
Score 10
Score 8
Score 11
Score 3
Score 8
Score 9
Score 5
Score 8
Score 10
Score 13
Score 10
Score 14
Score 12
Score 7
Score 8
Score 11
Score 8
Score 13
Score 6
Score 6
Score 6
Score 9
Score 7
Score 12
Score 10
Score 6
Score 6
Score 6
Score 5
Score 7
Score 8
Score 11
Score 10
Score 5
Score 10
Score 7
Score 9
Score 11
Score 8
Score 14
Score 6
Score 13
Score 12
Score 8
Score 6
Score 13
Score over time: 8.84
